In [1]:
from src.dataloading.finetuning_datasets import GTZANFinetuneDataset

GTZAN = GTZANFinetuneDataset(data_dir="/import/c4dm-datasets/gtzan_torchaudio/genres", target_length=5, n_augmentations=1)

/homes/jpmg86/anaconda3/envs/MuMRVQ/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from src.dataloading.finetuning_datasets import FineTuneDataModule

ftdm = FineTuneDataModule(train_data_dir="/import/c4dm-datasets/gtzan_torchaudio/genres", target_length=5, n_augmentations=1, batch_size=32, validation_split=0.1, num_workers=4)

In [5]:
ftdm.setup()

In [6]:
from tqdm import tqdm
dataloader = ftdm.train_dataloader()
for k in tqdm(range(len(dataloader))):
   print(next(iter(dataloader)))

{'wav': tensor([[[ 6.3448e-02, -5.4059e-02, -9.5746e-02,  ...,  5.3634e-02,
            6.1270e-02,  4.9721e-02]],
 
         [[-1.6919e-01, -1.2788e-01, -5.3676e-02,  ...,  5.0388e-05,
            1.5059e-02,  2.1972e-03]],
 
         [[ 3.3144e-02,  3.9009e-02,  4.0906e-02,  ..., -1.1315e-04,
            8.9326e-04,  1.6371e-03]],
 
         ...,
 
         [[ 3.1553e-01,  3.5091e-01,  3.1281e-01,  ..., -7.7571e-02,
           -2.5907e-02, -4.3251e-03]],
 
         [[-3.8526e-02, -3.5782e-02, -2.9526e-02,  ...,  2.9457e-02,
            2.4907e-02,  1.9358e-02]],
 
         [[ 2.1894e-01,  1.0481e-01,  9.1445e-02,  ...,  2.0602e-01,
            2.3675e-01,  1.7661e-01]]]),
 'label': tensor([4, 4, 1, 3, 4, 2, 6, 7, 1, 2, 4, 0, 6, 3, 4, 7, 9, 5, 6, 0, 6, 5, 7, 2,
         0, 6, 7, 8, 6, 6, 0, 2], dtype=torch.int8),
 'original_lens': tensor([120000, 120000, 120000, 120000, 120000, 120000, 120000, 120000, 120000,
         120000, 120000, 120000, 120000, 120000, 120000, 120000, 120000, 120

In [5]:
from src.models.finetune_model import MaCHUPFinetune
from src.models.model import MaCHUP
from src.models.Encodec import Encodec
from src.models.Transformer.Decoder import VanillaDecoder
from src.models.Transformer.Encoder import VanillaEncoder

machup = MaCHUP(encodec=Encodec(),encoder=VanillaEncoder(),decoder=VanillaDecoder(), adapt_sequence_len=True)

In [13]:
type(machup._modules.get('transformer_encoder'))

src.models.Transformer.Encoder.VanillaEncoder

In [3]:
import torch
from src.models import *

gtzanmodel = MaCHUPFinetune(encodec=Encodec(),encoder=VanillaEncoder(),decoder=VanillaDecoder(),target_layer="transformer_encoder", freeze_encoder=True)
test_data = torch.randn(2,1,1, 24000*5)
output = gtzanmodel(test_data)

tensor shape before augmentation batching: torch.Size([2, 1, 1, 120000])
tensor shape after augmentation batching: torch.Size([2, 1, 120000])
codes: torch.Size([2, 4, 375])
padding_mask: torch.Size([2, 375])
codes: torch.Size([2, 4, 375])
padding_mask: torch.Size([2, 375])
masking after with masking proba : 0.5
torch.Size([2, 375, 960])
torch.Size([960])
============== codes_masking ==============
tensor([[[ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False]],

        [[False,  True, False,  ..., False,  True, False],
         [False,  True, False,  ..., False,  True, False],
         [False,  True, False,  ..., False,  True, False],
         [False,  True, False,  ..., False,  True, False]]])
torch.Size([2, 4, 375])
1504 tokens were masked with random masking
tensor([[[1025,  732,  732,  ...,  399,  732,  399],
  

In [2]:
# check if all parameters are frozen in gtzanmodel.machup
for name, param in gtzanmodel.machup.named_parameters():
    assert param.requires_grad == False, name